# BENCHMARKING THE SOLVERS

Importing the relevant libraries:

In [1]:
import time
import numpy as np

In [2]:
# Including path to previous directory in built-in variable sys.path

import sys

sys.path.append('../')

In [3]:
# Importing features from the sudoku package

from sudoku.generators import generator
from sudoku.solvers import solver_backtracking_for_csp, solver_ip
from sudoku.utils import objective_grid, available_pos, available_nums

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


It was necessary to refactor solver_backtracking so that it stops when it finds the first solution, since it originally goes through the entire tree in order to check if there is more than one solution:

In [4]:
def solver_backtracking(sudoku_grid):
    grid = sudoku_grid.copy()  # does not change the original grid

    max_steps = np.count_nonzero(grid == 0)
    moves = [None] * max_steps
    actions = [None] * max_steps

    size = 0
    while not objective_grid(grid):
        # size == max_steps works like not available_pos(grid)
        while size == max_steps or not available_nums(grid, *available_pos(grid)[0]):
            size -= 1

            while not actions[size]:
                # There are no more possibilities for actions to be explored
                if size == 0:
                    return None

                x, y = moves[size]
                grid[y, x] = 0  # undoing each modification to generate the next successor
                size -= 1

            x, y = moves[size]
            num = actions[size].pop()

            grid[y, x] = num
            size += 1

        # Developing subtrees:
        # not at the search limit
        # there are actions available for the current tree level
        x, y = available_pos(grid)[0]
        nums = available_nums(grid, x, y)
        num = nums.pop()

        grid[y, x] = num
        moves[size] = (x, y)
        actions[size] = nums
        size += 1

    return grid

Implementing the benchmark function:

In [5]:
def solver_benchmark(func, args):  # receives a function and the arguments to be passed
    times = []
    
    for arg in args:
        start_time = time.time()
        
        # Checks if the solver was successful
        if not objective_grid(func(arg)):
            return None
        
        times.append(time.time() - start_time)
        
    return np.mean(times)  # returns the average resolution time for each instance

Defining the constants and auxiliary variables:

In [6]:
# Number of grids in the sample
N_SAMPLE = 20

# Times obtained
times_backtracking = []
times_backtracking_for_csp = []
times_ip = []

## Benchmarks using a sample with 41-clues grids

In [7]:
sample_41 = [generator() for _ in range(N_SAMPLE)]

sample_41

[array([[0, 0, 8, 5, 1, 6, 4, 9, 0],
        [6, 0, 1, 7, 3, 9, 0, 0, 2],
        [3, 0, 5, 0, 0, 2, 0, 6, 0],
        [0, 0, 6, 4, 0, 3, 0, 0, 7],
        [1, 0, 4, 0, 8, 7, 5, 0, 9],
        [0, 5, 0, 2, 9, 1, 0, 3, 4],
        [0, 0, 9, 0, 0, 0, 3, 0, 5],
        [4, 1, 0, 0, 6, 0, 0, 7, 0],
        [0, 0, 0, 3, 0, 0, 0, 1, 0]], dtype=int8),
 array([[8, 4, 6, 1, 0, 2, 3, 0, 5],
        [0, 5, 0, 9, 0, 0, 2, 8, 0],
        [0, 3, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 0, 4, 0, 1, 0, 0],
        [0, 0, 3, 2, 0, 0, 4, 5, 9],
        [0, 0, 8, 6, 0, 0, 7, 2, 0],
        [5, 8, 2, 0, 3, 6, 9, 0, 1],
        [0, 6, 4, 5, 1, 0, 0, 7, 0],
        [0, 9, 7, 4, 2, 0, 0, 3, 0]], dtype=int8),
 array([[0, 7, 3, 0, 1, 8, 0, 0, 0],
        [5, 0, 6, 0, 7, 0, 0, 0, 1],
        [4, 2, 1, 3, 9, 0, 0, 8, 5],
        [2, 0, 0, 0, 8, 7, 4, 0, 9],
        [1, 0, 0, 4, 2, 0, 0, 7, 0],
        [0, 0, 4, 0, 0, 5, 0, 0, 0],
        [0, 0, 0, 8, 0, 2, 0, 0, 0],
        [6, 5, 7, 9, 0, 0, 8, 1, 2],
        [8

In [8]:
times_backtracking.append(solver_benchmark(solver_backtracking, sample_41))

times_backtracking[0]

0.052167999744415286

In [9]:
times_backtracking_for_csp.append(solver_benchmark(solver_backtracking_for_csp, sample_41))

times_backtracking_for_csp[0]

0.07304555177688599

In [10]:
times_ip.append(solver_benchmark(solver_ip, sample_41))

times_ip[0]

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-365) rows, 0 (-729) columns and 0 (-2957) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-365) rows, 0 (-729) columns and 0 (-2957) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.08   (Wallclock seconds):       0.08

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-365) rows, 0 (-729) columns and 0 (-2957) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation

0.13559508323669434

## Benchmarks using a sample with 38-clues grids

In [11]:
sample_38 = [generator(38) for _ in range(N_SAMPLE)]

sample_38

[array([[0, 0, 2, 6, 3, 1, 9, 0, 7],
        [0, 7, 0, 0, 0, 9, 1, 0, 6],
        [0, 0, 0, 4, 0, 0, 0, 0, 0],
        [0, 0, 0, 9, 1, 3, 0, 7, 0],
        [0, 0, 9, 5, 2, 7, 8, 6, 1],
        [0, 0, 0, 0, 6, 0, 0, 0, 3],
        [9, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 3, 0, 9, 5, 6, 2, 0],
        [7, 4, 0, 1, 8, 2, 0, 3, 9]], dtype=int8),
 array([[8, 0, 5, 0, 0, 2, 3, 0, 0],
        [0, 2, 0, 0, 4, 6, 1, 5, 0],
        [0, 1, 0, 9, 0, 0, 7, 0, 0],
        [0, 5, 8, 0, 0, 0, 6, 9, 0],
        [3, 7, 6, 5, 2, 9, 0, 8, 0],
        [2, 4, 0, 1, 0, 8, 0, 0, 0],
        [5, 0, 0, 0, 0, 0, 9, 7, 0],
        [9, 0, 0, 0, 0, 4, 2, 1, 3],
        [0, 3, 0, 0, 0, 0, 8, 6, 0]], dtype=int8),
 array([[8, 0, 0, 3, 7, 1, 0, 0, 2],
        [0, 7, 0, 0, 0, 9, 0, 4, 0],
        [3, 0, 1, 0, 2, 4, 0, 0, 0],
        [0, 2, 6, 9, 8, 0, 3, 0, 0],
        [0, 0, 3, 0, 0, 0, 0, 6, 9],
        [7, 0, 0, 5, 0, 0, 8, 2, 1],
        [2, 0, 5, 4, 0, 3, 9, 7, 0],
        [0, 0, 7, 0, 1, 8, 0, 0, 0],
        [0

In [12]:
times_backtracking.append(solver_benchmark(solver_backtracking, sample_38))

times_backtracking[1]

0.0901032567024231

In [13]:
times_backtracking_for_csp.append(solver_benchmark(solver_backtracking_for_csp, sample_38))

times_backtracking_for_csp[1]

0.06727715730667114

In [14]:
times_ip.append(solver_benchmark(solver_ip, sample_38))

times_ip[1]

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.08   (Wallclock seconds):       0.08

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-362) rows, 0 (-729) columns and 0 (-2954) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-362) rows, 0 (-729) columns and 0 (-2954) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation

0.08990638256072998

## Benchmarks using a sample with 35-clues grids

In [15]:
sample_35 = [generator(35) for _ in range(N_SAMPLE)]

sample_35

[array([[6, 0, 0, 0, 4, 0, 2, 0, 0],
        [0, 0, 3, 6, 0, 7, 0, 0, 9],
        [0, 5, 0, 0, 9, 1, 0, 0, 0],
        [0, 0, 2, 0, 0, 9, 0, 0, 0],
        [9, 6, 1, 0, 7, 0, 0, 3, 0],
        [0, 0, 8, 0, 2, 6, 5, 0, 0],
        [1, 0, 7, 0, 0, 0, 0, 0, 5],
        [3, 9, 5, 0, 6, 4, 0, 0, 8],
        [8, 0, 0, 5, 1, 2, 0, 7, 0]], dtype=int8),
 array([[6, 5, 0, 4, 0, 0, 7, 9, 3],
        [0, 0, 0, 0, 3, 0, 0, 5, 0],
        [0, 0, 0, 2, 6, 0, 4, 1, 0],
        [8, 0, 0, 0, 0, 0, 5, 4, 9],
        [0, 0, 0, 1, 0, 0, 3, 8, 0],
        [0, 4, 6, 0, 5, 9, 0, 0, 0],
        [1, 0, 4, 0, 2, 0, 8, 0, 5],
        [0, 0, 3, 0, 9, 0, 0, 7, 0],
        [2, 0, 0, 5, 1, 8, 0, 0, 0]], dtype=int8),
 array([[0, 0, 0, 8, 3, 6, 9, 2, 0],
        [8, 2, 0, 4, 0, 0, 0, 0, 5],
        [6, 0, 9, 5, 0, 0, 4, 0, 8],
        [2, 0, 0, 0, 4, 0, 0, 6, 0],
        [3, 0, 4, 6, 0, 0, 0, 5, 0],
        [0, 0, 0, 0, 0, 5, 0, 8, 4],
        [0, 0, 2, 0, 0, 8, 0, 0, 0],
        [9, 0, 8, 0, 6, 0, 0, 1, 0],
        [5

In [16]:
times_backtracking.append(solver_benchmark(solver_backtracking, sample_35))

times_backtracking[2]

0.3546122670173645

In [17]:
times_backtracking_for_csp.append(solver_benchmark(solver_backtracking_for_csp, sample_35))

times_backtracking_for_csp[2]

0.0945162057876587

In [18]:
times_ip.append(solver_benchmark(solver_ip, sample_35))

times_ip[2]

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-359) rows, 0 (-729) columns and 0 (-2951) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-359) rows, 0 (-729) columns and 0 (-2951) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation

0.10399413108825684

## Benchmark for a specific sudoku instance

The following instance was created by Finnish mathematician Arto Inkala and is considered to be the hardest sudoku game in the world. On a difficulty scale ranging from 1 to 5 stars, the instance has 11 stars!

In [19]:
grid_inkala = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                        [8, 0, 0, 0, 0, 0, 0, 2, 0],
                        [0, 7, 0, 0, 1, 0, 5, 0, 0],
                        [4, 0, 0, 0, 0, 5, 3, 0, 0],
                        [0, 1, 0, 0, 7, 0, 0, 0, 6],
                        [0, 0, 3, 2, 0, 0, 0, 8, 0],
                        [0, 6, 0, 5, 0, 0, 0, 0, 9],
                        [0, 0, 4, 0, 0, 0, 0, 3, 0],
                        [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

In [20]:
times_backtracking.append(solver_benchmark(solver_backtracking, [grid_inkala]))

times_backtracking[3]

32.745331048965454

In [21]:
times_backtracking_for_csp.append(solver_benchmark(solver_backtracking_for_csp, [grid_inkala]))

times_backtracking_for_csp[3]

0.17348933219909668

In [22]:
times_ip.append(solver_benchmark(solver_ip, [grid_inkala]))

times_ip[3]

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.07   (Wallclock seconds):       0.07

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 189 (-158) rows, 185 (-544) columns and 760 (-2179) elements
Clp1000I sum of infeasibilities 3.43475e-06 - average 1.81733e-08, 7 fixed columns
Coin0506I Presolve 173 (-16) rows, 168 (-17) columns and 709 (-51) elements
Clp0006I 0  Obj 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I O

2.5007643699645996

## Summarizing the data obtained

In [23]:
# Indexes, columns and results obtained
index = ['backtracking', 'backtracking_for_csp', 'ip']
columns = ['41_clues', '38_clues', '35_clues', 'grid_inkala']
results = [times_backtracking, times_backtracking_for_csp, times_ip]

# Auxiliary variables to perform indentation
indent_index = max(map(len, index)) + 1
indent_column = min(map(len, columns))

In [24]:
print(' ' * indent_index, '41_clues', '38_clues', '35_clues', 'grid_inkala', sep='|')
print('-'*61)

for solver_name, result in zip(index, results):
    print(f'{solver_name:{indent_index}}', end='|')
    
    for time in result[:-1]:
        print(f'{str(time):.{indent_column}}', end='|')
    print(f'{str(result[-1]):.{indent_column}}')
    
    print('-'*61)

                     |41_clues|38_clues|35_clues|grid_inkala
-------------------------------------------------------------
backtracking         |0.052167|0.090103|0.354612|32.74533
-------------------------------------------------------------
backtracking_for_csp |0.073045|0.067277|0.094516|0.173489
-------------------------------------------------------------
ip                   |0.135595|0.089906|0.103994|2.500764
-------------------------------------------------------------


By observing the data we can see that the regular search method with backtracking is very efficient for easier sudoku instances, however it becomes unfeasible to solve more complex instances because the execution time increases considerably.

Besides solving sudoku through integer linear programming is a good option, the search method for constraint satisfaction problems proves to be the most suitable. In this search strategy, the resolution time remains below two tenths of a second for the most difficult instance in the game.